# Syosset Park PUMS Data Analysis

CGR was asked to do an analysis of the number of school children likely to be added to a community due to a housing development.  We used the 2011-15 ACS data in order to estimate the impact.

# Import the Data
We have downloaded and extracted the 2011-15 PUMS data from zip files found at https://www2.census.gov/programs-surveys/acs/data/pums/2015/5-Year/.

In [1]:
import pandas as pd
import numpy as np

# Read in the housing data
df_h = pd.read_csv('ss15hny.csv', low_memory=False)
# Present a sample of the data
df_h.iloc[:,:15].head()

,insp,RT,SERIALNO,DIVISION,PUMA00,PUMA10,REGION,ST,ADJHSG,ADJINC,WGTP,NP,TYPE,ACR,AGS
0,NaN,H,2011000000003,2,2602,-9,1,36,1053874,1073094,8,1,1,NaN,NaN
1,NaN,H,2011000000017,2,4309,-9,1,36,1053874,1073094,16,1,1,NaN,NaN
2,NaN,H,2011000000023,2,3502,-9,1,36,1053874,1073094,0,1,2,NaN,NaN
3,500.0,H,2011000000057,2,803,-9,1,36,1053874,1073094,11,4,1,2.0,1.0
4,700.0,H,2011000000063,2,4110,-9,1,36,1053874,1073094,28,1,1,NaN,NaN


In [2]:
# Read in the person data
df_p = pd.read_csv('ss15pny.csv')
# Present a sample of the data
df_p.iloc[:,:15].head()

E:\Python3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,RT,SERIALNO,SPORDER,PUMA00,PUMA10,ST,ADJINC,PWGTP,AGEP,CIT,CITWP05,CITWP12,COW,DDRS,DEAR
0,P,2011000000003,1,2602,-9,36,1073094,8,70,1,NaN,NaN,NaN,2.0,2
1,P,2011000000017,1,4309,-9,36,1073094,17,69,4,1995.0,-9.0,1.0,2.0,1
2,P,2011000000023,1,3502,-9,36,1073094,13,81,1,NaN,NaN,NaN,2.0,2
3,P,2011000000057,1,803,-9,36,1073094,11,46,1,NaN,NaN,4.0,2.0,2
4,P,2011000000057,2,803,-9,36,1073094,8,47,1,NaN,NaN,6.0,2.0,2


However in an effort to conserve computer resources we are only going to keep a subset of all the varriables

In [3]:
# These are the people variables we want to keep 
p_vars = ['SERIALNO', 'AGEP', 'PUMA00', 'PUMA10', 'PWGTP', 'PWGTP1', 'PWGTP2',
          'PWGTP3', 'PWGTP4', 'PWGTP5', 'PWGTP6', 'PWGTP7', 'PWGTP8', 'PWGTP9',
          'PWGTP10', 'PWGTP11', 'PWGTP12', 'PWGTP13', 'PWGTP14', 'PWGTP15',
          'PWGTP16', 'PWGTP17', 'PWGTP18', 'PWGTP19', 'PWGTP20', 'PWGTP21',
          'PWGTP22', 'PWGTP23', 'PWGTP24', 'PWGTP25', 'PWGTP26', 'PWGTP27',
          'PWGTP28', 'PWGTP29', 'PWGTP30', 'PWGTP31', 'PWGTP32', 'PWGTP33',
          'PWGTP34', 'PWGTP35', 'PWGTP36', 'PWGTP37', 'PWGTP38', 'PWGTP39',
          'PWGTP40', 'PWGTP41', 'PWGTP42', 'PWGTP43', 'PWGTP44', 'PWGTP45',
          'PWGTP46', 'PWGTP47', 'PWGTP48', 'PWGTP49', 'PWGTP50', 'PWGTP51',
          'PWGTP52', 'PWGTP53', 'PWGTP54', 'PWGTP55', 'PWGTP56', 'PWGTP57',
          'PWGTP58', 'PWGTP59', 'PWGTP60', 'PWGTP61', 'PWGTP62', 'PWGTP63',
          'PWGTP64', 'PWGTP65', 'PWGTP66', 'PWGTP67', 'PWGTP68', 'PWGTP69',
          'PWGTP70', 'PWGTP71', 'PWGTP72', 'PWGTP73', 'PWGTP74', 'PWGTP75',
          'PWGTP76', 'PWGTP77', 'PWGTP78', 'PWGTP79', 'PWGTP80']
# Subset the data frame
df_p = df_p[p_vars]
# Present the data again
df_p.iloc[:,:15].head()

,SERIALNO,AGEP,PUMA00,PUMA10,PWGTP,PWGTP1,PWGTP2,PWGTP3,PWGTP4,PWGTP5,PWGTP6,PWGTP7,PWGTP8,PWGTP9,PWGTP10
0,2011000000003,70,2602,-9,8,7,2,12,8,2,3,8,9,7,8
1,2011000000017,69,4309,-9,17,5,24,10,24,34,13,22,17,35,4
2,2011000000023,81,3502,-9,13,2,2,12,12,13,2,23,2,13,23
3,2011000000057,46,803,-9,11,10,3,3,18,3,10,11,16,20,20
4,2011000000057,47,803,-9,8,8,2,2,13,3,6,10,10,14,15


# Method/Function
When working with PUMS data the individual must filter the data based on the criterias of interest and sum up the weight in order to produce a point estimate.  The https://www2.census.gov/programs-surveys/acs/tech_docs/pums/ACS2011_2015_PUMS_README.pdf explains how to estimate the standard error and margin of error of the point estimate.  We will create one function that will take the Pandas data frame and calculate all three measures using a specified weight.


In [4]:
def get_estimates_and_errors(df, weight):
    """Calculates the estimate, standard error and margin of error of PUMS data

    Args:
        df (DataFrame): the pandas data frame of PUMS data.
        weight (str): the weighting column name (WGTP or PWGTP).
    Returns:
        estimate (int): The point estimate.
        se (int): The standard error of the estimate.
        moe (int): The margin of error of the estimate.
    """
    # Translate the weight into standard error weights 
    # i.e. 'WGPT' to ['WGTP1', ..., 'WGTP80']
    se_weights = [weight+str(i) for i in range(1,81)]
    # Calculate the point estimate
    estimate = df[[weight]].sum()[weight]
    # Calculate the standard error 
    se = ((4/80)*((df[se_weights].sum() - estimate)**2).sum())**0.5
    # Calculate the margin of error
    moe = 1.645 * se
    # Convert to integers
    estimate = int(estimate)
    se = int(round(se,0))
    moe = int(round(moe,0))
    # Return the values
    return estimate, se, moe

# Verify the Data & Method/Function
In order to verify our process we will first do some calculations and compare them against the values generated for NYS.  We used the PUMS estimates provided by the Census Bureau (https://www2.census.gov/programs-surveys/acs/tech_docs/pums/estimates/) and custom tabulations using Data Ferrett (https://dataferrett.census.gov/) as the basis of comparison.

## PUMS Estimates
### Total Population
Let's see if the function defined above an duplicate the results in the PUMS estimates.  If the function is working properly the following should produce:

Estimate: 19,673,174<br/>
Standard Error: 0<br/>
Margin of Error: 0

In [5]:
estimate, se, moe = get_estimates_and_errors(df_p, 'PWGTP')
print('Estimate: '+str(estimate))
print('Standard Error: '+str(se))
print('Margin of Error: '+str(moe))

Estimate: 19673174
Standard Error: 0
Margin of Error: 0


### Owner occupied units (TEN in 1,2)
Now turning our focus to the housing data let's check if it produces the following:

Estimate: 3,877,343<br/>
Standard Error: 10,932<br/>
Margin of Error: 17,983

In [6]:
tenure_filter = np.logical_or(df_h.TEN == 1, df_h.TEN == 2)
estimate, se, moe = get_estimates_and_errors(df_h[tenure_filter], 'WGTP')
print('Estimate: '+str(estimate))
print('Standard Error: '+str(se))
print('Margin of Error: '+str(moe))

Estimate: 3877343
Standard Error: 10932
Margin of Error: 17983


### Vacant units (VACS has value)
We'll check a second variable against the PUMS estimates.  If it is working it should return the following:

Estimate: 909,441<br/>
Standard Error: 7,185<br/>
Margin of Error: 11,820

In [7]:
vacancy_filter = np.logical_not(np.isnan(df_h.VACS))
estimate, se, moe = get_estimates_and_errors(df_h[vacancy_filter], 'WGTP')
print('Estimate: '+str(estimate))
print('Standard Error: '+str(se))
print('Margin of Error: '+str(moe))

Estimate: 909441
Standard Error: 7185
Margin of Error: 11820


## Data Ferrett Estimates
Having successfully duplicated the published estimates we will move on to some more complicated estimates that we produced using data ferrett.
### Number of Bedrooms
We will be producing estimates based on the number of bedrooms so let's see if we can reproduce what Data Ferrett produced.  Here are the counts by the number of bedrooms:

    0 = 364,738
    1 = 1,608,898
    2 = 2,188,920
    3 = 2,570,555
    4 = 1,086,802
    5 = 255,888
    6 = 14,041
    7 = 0
    8 = 62,118
    9 = 19,765
    N/A (GQ) = 0
    
We will recode these variables and will create a 6+ category, which value should be 95,924.  Data Ferrett currently does not (as far as I am aware) produce standard errors or the margin of error.  But since we have proven the methodology using the published PUMS estimates we can be sure that what it generates will be accurate.

Let's recode the number of bedrooms variable.  From previous exploratory analysis I know that the values of the number of bedrooms range from 0 to 9.  The results may be in a different sort order than presented above.

In [8]:
def recode_BDSP(val):
    """Recodes the number of bedrooms (BDSP) PUMS variable

    Args:
        val (mixed): the value to be recoded.
    Returns:
        recoded_val (str): The new categorical variable.
    """
    # This is the mapping of values to the recoded value
    code = {0:'Studio', 1:'1 Bedroom', 2:'2 Bedrooms', 3:'3 Bedrooms', 4:'4 Bedrooms', 5:'5 Bedrooms', 
            6:'6+ Bedrooms',  7:'6+ Bedrooms', 8:'6+ Bedrooms', 9:'6+ Bedrooms'}
    # Get the recoded value and return it
    if val not in code:
        recoded_val = 'N/A (GQ)'
    else:
        recoded_val = code[val]
    return recoded_val

# Recode the number of bedrooms
df_h['n_bedrooms'] = df_h.BDSP.apply(recode_BDSP, 0)

# Produce estimates for each number
ns_bedrooms = df_h.n_bedrooms.unique()
# Sort the list
ns_bedrooms.sort()
# Step through the list item by item
for n_bedrooms in ns_bedrooms:
    # Create a filter for the data frame
    df_filter = df_h.n_bedrooms == n_bedrooms
    # Generate the estimates
    estimate, se, moe = get_estimates_and_errors(df_h[df_filter], 'WGTP')
    # Display the results
    print('Estimate of '+str(n_bedrooms)+' units: '+str(estimate)+' +/- '+str(moe))

Estimate of 1 Bedroom units: 1608898 +/- 10703
Estimate of 2 Bedrooms units: 2188920 +/- 10909
Estimate of 3 Bedrooms units: 2570555 +/- 11081
Estimate of 4 Bedrooms units: 1086802 +/- 7778
Estimate of 5 Bedrooms units: 255888 +/- 3839
Estimate of 6+ Bedrooms units: 95924 +/- 2420
Estimate of N/A (GQ) units: 0 +/- 0
Estimate of Studio units: 364738 +/- 6594


### People by Number of Bedrooms
So far we have been evaluating the data in isolation.  Let's see what happens when we put them together.  We want the count of people by number of bedrooms.  According to data ferrett, in NYS this is how it breaks out:

    0 = 424,789	
    1 = 2,374,676
    2 = 4,652,957
    3 = 6,872,722
    4 = 3,445,711
    5 = 941,339
    6 = 51,239
    7 = 0
    8 = 252,461
    9 = 78,025
    N/A (GQ) = 579,255
    
Let's see what happens:

In [9]:
# To free up some memory we need to subset the housing data
h_vars = ['SERIALNO', 'n_bedrooms', 'VALP', 'ADJINC', 'TEN', 
          'WGTP', 'WGTP1', 'WGTP2', 'WGTP3', 'WGTP4', 'WGTP5', 
          'WGTP6', 'WGTP7', 'WGTP8', 'WGTP9', 'WGTP10', 'WGTP11', 
          'WGTP12', 'WGTP13', 'WGTP14', 'WGTP15', 'WGTP16', 'WGTP17', 
          'WGTP18', 'WGTP19', 'WGTP20', 'WGTP21', 'WGTP22', 'WGTP23', 
          'WGTP24', 'WGTP25', 'WGTP26', 'WGTP27', 'WGTP28', 'WGTP29', 
          'WGTP30', 'WGTP31', 'WGTP32', 'WGTP33', 'WGTP34', 'WGTP35', 
          'WGTP36', 'WGTP37', 'WGTP38', 'WGTP39', 'WGTP40', 'WGTP41', 
          'WGTP42', 'WGTP43', 'WGTP44', 'WGTP45', 'WGTP46', 'WGTP47', 
          'WGTP48', 'WGTP49', 'WGTP50', 'WGTP51', 'WGTP52', 'WGTP53', 
          'WGTP54', 'WGTP55', 'WGTP56', 'WGTP57', 'WGTP58', 'WGTP59', 
          'WGTP60', 'WGTP61', 'WGTP62', 'WGTP63', 'WGTP64', 'WGTP65', 
          'WGTP66', 'WGTP67', 'WGTP68', 'WGTP69', 'WGTP70', 'WGTP71', 
          'WGTP72', 'WGTP73', 'WGTP74', 'WGTP75', 'WGTP76', 'WGTP77', 
          'WGTP78', 'WGTP79', 'WGTP80']

# Now we can join the two data frames together (INNER JOIN in SQL speak)
df = pd.merge(df_h[h_vars], df_p, on='SERIALNO')

# Cross your fingers and hope this works
ns_bedrooms = df.n_bedrooms.unique()
ns_bedrooms.sort()
for n_bedrooms in ns_bedrooms:
    df_filter = df.n_bedrooms == n_bedrooms
    estimate, se, moe = get_estimates_and_errors(df[df_filter], 'PWGTP')
    print('Estimate for '+str(n_bedrooms)+': '+str(estimate)+' +/- '+str(moe))

Estimate for 1 Bedroom: 2374676 +/- 18148
Estimate for 2 Bedrooms: 4652957 +/- 29125
Estimate for 3 Bedrooms: 6872722 +/- 31345
Estimate for 4 Bedrooms: 3445711 +/- 25370
Estimate for 5 Bedrooms: 941339 +/- 17266
Estimate for 6+ Bedrooms: 381725 +/- 12280
Estimate for N/A (GQ): 579255 +/- 0
Estimate for Studio: 424789 +/- 9586


Now that we have verified the methodology we can proceed with the analysis

# Syosset Park Analysis 

## Variable Recoding

We will estimate the number of people and school age children resulting from the housing development.  Our model will use two different data points.  One is the number of bedrooms and the other is the value of the housing.  We have the number of bedrooms recoded but we don't have the home value.

Since the PUMS data spans a 5 year period it must undergo an inflation adjustement to bring it into 2015 dollars.  This will be preformed then the data will be recoded based on the 2015 dollar value of the property.

In [10]:
# The recode home value function
def recode_inf_adj_val(val):
    """Recodes the dollar value into one of our categories

    Args:
        val (mixed): the value to be recoded.
    Returns:
        recoded_val (str): The new categorical variable.
    """
    if np.isnan(val):   return('N/A (GQ)')
    elif val < 100000:  return('Less than $100,000')
    elif val < 200000:  return("$100,000's")
    elif val < 300000:  return("$200,000's")
    elif val < 400000:  return("$300,000's")
    elif val < 500000:  return("$400,000's")
    elif val < 600000:  return("$500,000's")
    elif val < 700000:  return("$600,000's")
    elif val < 800000:  return("$700,000's")
    elif val < 900000:  return("$800,000's")
    elif val < 1000000: return("$900,000's")
    else: return('$1,000,000 or Higher')

# This is the inflation adjustment
df['inf_adj_val'] = df.VALP * (df.ADJINC/1000000)

# Now we can recode the home value
df['home_val'] = df.inf_adj_val.apply(recode_inf_adj_val, 0)

The last variable that will need to be recoded is a flag for if the individual is a school age child or not.  We are defining school age as 5-17 years old.

In [11]:
# The recode home value function
def recode_AGEP(val):
    """Recodes the dollar value into one of our categories

    Args:
        val (mixed): the value to be recoded.
    Returns:
        recoded_val (str): The new categorical variable.
    """
    if np.isnan(val):   return('N/A')
    elif val > 4 and val < 18:  return('Yes')
    else: return('No')

# Now we can recode the home value
df['school_age_child'] = df.AGEP.apply(recode_AGEP, 0)

## Filtering the Data

We only are interested in a subset of our data frame.  

### Geography

Geographically we are only looking at the Syosset School District.

In [12]:
# The following PUMAS are what we want to include
valid_PUMA00 = ['04202', '04203']
valid_PUMA10 = ['03202', '03203']

# Create the geography filter
geo_filter = np.logical_or(df['PUMA00'].isin(valid_PUMA00), df['PUMA10'].isin(valid_PUMA10))

### Tenure

Since the development will be condos we are only interested in the population living in owner-occupied housing.

In [13]:
tenure_filter = np.logical_or(df['TEN']==1, df['TEN'] == 2)

### Final Filter
We can bring the two together for our data frame filter

In [14]:
df_filter = np.logical_and(geo_filter, tenure_filter)

# Get the row and column count before filtering
df.shape

(981671, 173)

In [15]:
# Filter the data
df = df[df_filter]

# Get the row and column count after filtering
df.shape

(10598, 173)

## Total Housing Units by Type Estimates
In order to get the count of housing units by the number of bed rooms and home value we will need to apply the same filters and recoding.

In [16]:
# Create the geography filter
geo_filter = np.logical_or(df_h['PUMA00'].isin(valid_PUMA00), df_h['PUMA10'].isin(valid_PUMA10))
# Create the tenure filter
tenure_filter = np.logical_or(df_h['TEN']==1, df_h['TEN'] == 2)
# Combine the two
df_filter = np.logical_and(geo_filter, tenure_filter)
# Filter the data
df_h = df_h[df_filter]

### By Number of Bedrooms

In [17]:
ns_bedrooms = df_h.n_bedrooms.unique()
ns_bedrooms.sort()
for n_bedrooms in ns_bedrooms:
    df_filter = df_h.n_bedrooms == n_bedrooms
    estimate, se, moe = get_estimates_and_errors(df_h[df_filter], 'WGTP')
    print('Estimate for '+str(n_bedrooms)+': '+str(estimate)+' +/- '+str(moe))

Estimate for 1 Bedroom: 1152 +/- 303
Estimate for 2 Bedrooms: 5303 +/- 477
Estimate for 3 Bedrooms: 25801 +/- 944
Estimate for 4 Bedrooms: 19019 +/- 972
Estimate for 5 Bedrooms: 6570 +/- 664
Estimate for 6+ Bedrooms: 1855 +/- 320
Estimate for Studio: 28 +/- 34


### By Home Value

In [18]:
# This is the inflation adjustment
df_h['inf_adj_val'] = df_h.VALP * (df_h.ADJINC/1000000)

# Now we can recode the home value
df_h['home_val'] = df_h.inf_adj_val.apply(recode_inf_adj_val, 0)

home_vals = df_h.home_val.unique()
home_vals.sort()
for home_val in home_vals:
    df_filter = df_h.home_val == home_val
    estimate, se, moe = get_estimates_and_errors(df_h[df_filter], 'WGTP')
    print('Estimate for '+str(home_val)+': '+str(estimate)+' +/- '+str(moe))

Estimate for $1,000,000 or Higher: 7773 +/- 501
Estimate for $100,000's: 1224 +/- 279
Estimate for $200,000's: 1968 +/- 332
Estimate for $300,000's: 9525 +/- 667
Estimate for $400,000's: 13724 +/- 859
Estimate for $500,000's: 9111 +/- 632
Estimate for $600,000's: 6879 +/- 509
Estimate for $700,000's: 3529 +/- 455
Estimate for $800,000's: 2533 +/- 374
Estimate for $900,000's: 1728 +/- 268
Estimate for Less than $100,000: 1734 +/- 305


## Total People by Housing Type Estimates
Now we will repeat the process on our merged data

### By Number of Bedrooms

In [19]:
for n_bedrooms in ns_bedrooms:
    df_filter = df.n_bedrooms == n_bedrooms
    estimate, se, moe = get_estimates_and_errors(df[df_filter], 'PWGTP')
    print('Estimate for '+str(n_bedrooms)+': '+str(estimate)+' +/- '+str(moe))

Estimate for 1 Bedroom: 1553 +/- 407
Estimate for 2 Bedrooms: 9912 +/- 958
Estimate for 3 Bedrooms: 73130 +/- 2966
Estimate for 4 Bedrooms: 62504 +/- 3312
Estimate for 5 Bedrooms: 23320 +/- 2390
Estimate for 6+ Bedrooms: 7548 +/- 1448
Estimate for Studio: 42 +/- 54


### By Home Value

In [20]:
for home_val in home_vals:
    df_filter = df.home_val == home_val
    estimate, se, moe = get_estimates_and_errors(df[df_filter], 'PWGTP')
    print('Estimate for '+str(home_val)+': '+str(estimate)+' +/- '+str(moe))

Estimate for $1,000,000 or Higher: 25378 +/- 1587
Estimate for $100,000's: 2159 +/- 613
Estimate for $200,000's: 3781 +/- 745
Estimate for $300,000's: 26001 +/- 2151
Estimate for $400,000's: 41063 +/- 3159
Estimate for $500,000's: 28600 +/- 2080
Estimate for $600,000's: 21572 +/- 2014
Estimate for $700,000's: 11103 +/- 1656
Estimate for $800,000's: 8442 +/- 1318
Estimate for $900,000's: 5510 +/- 1033
Estimate for Less than $100,000: 4400 +/- 845


## Total School Age Children by Housing Type Estimates
Now we will repeat the process on our merged data

### By Number of Bedrooms

In [21]:
for n_bedrooms in ns_bedrooms:
    df_filter = np.logical_and(df.n_bedrooms == n_bedrooms, df.school_age_child == 'Yes')
    estimate, se, moe = get_estimates_and_errors(df[df_filter], 'PWGTP')
    print('Estimate for '+str(n_bedrooms)+': '+str(estimate)+' +/- '+str(moe))

Estimate for 1 Bedroom: 38 +/- 49
Estimate for 2 Bedrooms: 611 +/- 232
Estimate for 3 Bedrooms: 11778 +/- 1004
Estimate for 4 Bedrooms: 12293 +/- 1094
Estimate for 5 Bedrooms: 4795 +/- 726
Estimate for 6+ Bedrooms: 1588 +/- 422
Estimate for Studio: 0 +/- 0


### By Home Value

In [22]:
for home_val in home_vals:
    df_filter = np.logical_and(df.home_val == home_val, df.school_age_child == 'Yes')
    estimate, se, moe = get_estimates_and_errors(df[df_filter], 'PWGTP')
    print('Estimate for '+str(home_val)+': '+str(estimate)+' +/- '+str(moe))

Estimate for $1,000,000 or Higher: 5545 +/- 658
Estimate for $100,000's: 117 +/- 93
Estimate for $200,000's: 196 +/- 137
Estimate for $300,000's: 3261 +/- 629
Estimate for $400,000's: 6784 +/- 1006
Estimate for $500,000's: 4940 +/- 784
Estimate for $600,000's: 4219 +/- 770
Estimate for $700,000's: 2536 +/- 670
Estimate for $800,000's: 1804 +/- 439
Estimate for $900,000's: 1243 +/- 395
Estimate for Less than $100,000: 458 +/- 282
